In [1]:
import pandas as pd
import numpy as np
import json

In [5]:
bw=pd.read_csv('bw_w9.csv')
ci=pd.read_csv('ci_w9.csv')
match3=['fuma costume',
 'alim',
 'i and j hcm',
 'j and d vinako',
 'long thanh branch',
 'roo hsing co. nicaragua sa',
 'crystal martin',
 'lee and suy vina',
 'phuong nam garment trading import export jsc',
 'phu tai linh service and trading',
 'w and d apparel corp',
 'avery dennison group',
 'new century clothing industry llc',
 'eins',
 'united apparel cambodia branch 2 (formerly lotus textile garment cambodia  branch 2)',
 'dha',
 'yi da viet nam ltd',
 'j t h textile (cambodia) inc. (formerly united apparel (cambodia) 3, formerly lotus 3',
 'elite',
 'park corp ltd',
 'g h i garments cambodia',
 'gildan activewear rivas ii, sa',
 'zhong yin b textile',
 'seil',
 'tainan enterprises',
 'casual wear apparel llc',
 'hong kong winfair garment ltd',
 'emperor',
 'fashion garment 2',
 'opus one corp',
 'quint major industrial vietnam',
 'atto',
 'jaya perkasa textile pt',
 's w j garment',
 'southern garment manufacturing',
 'evergreen apparel',
 'yu fa garment industry',
 'my viet international export service trade',
 'e m b jeilkovi',
 'my hung garment and trading jsc',
 'a and j carter ltd',
 'premier global garment factory',
 't and k garment industry',
 'nesia pan pacific clothing pt',
 'yu da garment industry (cambodia)  (formerly maxpearl garment)',
 'indo jordan clothing company',
 'ty fashion (cambodia) plc. (no.6) (formerly t y fashion (cambodia)  (branch 2)',
 'nesia pan pacific knit pt',
 'm and j garment-printing-embroidery',
 'hai duong garment stock company no.2',
 'dress shirts ltd',
 'h and h textiles, sa',
 'youngone nam dinh',
 'smart shirts enterprise ltd',
 'casual wear apparel',
 'xuan hoa garment',
 'van thanh trading service',
 'h and l apparel corp',
 'g and g ii garments',
 'fashion garments 2',
 'international trading services ltd',
 'rina jaya garment pt',
 'ty fashion (cambodia) plc. (no.5) (formerly t y fashion (cambodia) )',
 'j t h textile inc',
 'd and blue',
 'pan pacific jakarta cabang semarang pt',
 'ty fashion (cambodia) plc. (formerly no.4 and no.5)',
 'leading star garment',
 'dong a textile garment jsc',
 'nam and co london',
 'classic fashion concept ltd',
 'g and g fashion',
 'tan uy dat',
 'laspo pt',
 'dae kwang apparel jsc']

In [53]:
def getFirst2(s):
    if len(s.split(" "))<2:
        return s
    
    op=" ".join(s.split(" ")[:2])
    if len(s.split(" "))>2:
        op=op+" "
    return op

def ciInfoGet(cName):
    global ci
    return ci[ci.cName==cName]


In [30]:
for i in match3:
    bwc=bw[bw.cName==i].Country.unique()[0]
    poss=ci[(ci.cName.str.contains(getFirst2(i))) & (ci.headquarters_country==bwc) ].cName.unique()
    if len(poss)==0:
        continue
    print("#",i," : ",poss)

# fuma costume  :  ['new fuma costume']
# i and j hcm  :  ['rai and rohl technologies inc'
 'thuong mai and khai thac khoang san duong hieu ctcp']
# long thanh branch  :  ['sonadezi long thanh shareholding co' 'long thanh plastic']
# lee and suy vina  :  ['vietnam lee and man paper manufacturing ltd']
# phuong nam garment trading import export jsc  :  ['phuong nam cultural jsc'
 'phuong nam education investment and development jsc']
# phu tai linh service and trading  :  ['phu tai jsc']
# elite  :  ['green elite group']
# gildan activewear rivas ii, sa  :  ['gildan activewear san marcos ii sa']
# zhong yin b textile  :  ['zhong yin cambodia b textile']
# jaya perkasa textile pt  :  ['jatim jaya perkasa pt' 'pakindo jaya perkasa pt' 'mitra jaya perkasa cv'
 'arumba jaya perkasa pt' 'nusa jaya perkasa pt'
 'rajawali jaya perkasa pt' 'solusi jaya perkasa pt'
 'munte waniq jaya perkasa pt']
# evergreen apparel  :  ['sd evergreen apparel']
# my hung garment and trading jsc  :  ['phu my hung

In [68]:
len(bw[bw.repriskID.isna()])

4477

In [67]:
with open('m3.txt') as m3:
    m3=m3.read().split("\n")
    m3.remove("")

m3
for i in m3:
    i=i.replace("#","")
    # .replace("[","").replace("]","").replace("'","")
    doubt = 1 if i.startswith("*") else 0
    i=i.replace("*","")
    bwName=i.split(":")[0].strip()
    ciNames=eval(i.split(":")[1].strip()) #list type
    # print(bwName,ciNames,doubt)
    if len(ciNames)==1:
        ciNameCorr=ciNames[0]
        bw.loc[bw.cName==bwName, "CNRR"]=ci.loc[ci.cName==ciNameCorr,"company_name"].values[0]
        bw.loc[bw.cName==bwName, "repriskID"]=ci.loc[ci.cName==ciNameCorr,"repriskID"].values[0]
        bw.loc[bw.cName==bwName, "RRCleaned"]=ciNameCorr
        bw.loc[bw.cName==bwName, "RRSector"]=ci.loc[ci.cName==ciNameCorr,"sectors"].values[0]
        bw.loc[bw.cName==bwName, "doubt"]=doubt

In [69]:
bw

,QuestID,FactoryBWID,FactoryName,Country,AssessmentDate,year,FNC,cName,CNRR,repriskID,RRCleaned,RRSector,doubt
0,12129,2,Hi-Tech Textile L.L.C.,Jordan,19/10/2015,2015,NaN,hi-tech textile llc,Hi-Tech Textile LLC,106981.0,hi-tech textile llc,Personal and Household Goods,0
1,17215,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2016,2016,NaN,hi-tech textile llc,Hi-Tech Textile LLC,106981.0,hi-tech textile llc,Personal and Household Goods,0
2,22596,2,Hi-Tech Textile L.L.C.,Jordan,27/09/2017,2017,NaN,hi-tech textile llc,Hi-Tech Textile LLC,106981.0,hi-tech textile llc,Personal and Household Goods,0
3,30133,2,Hi-Tech Textile L.L.C.,Jordan,16/10/2018,2018,NaN,hi-tech textile llc,Hi-Tech Textile LLC,106981.0,hi-tech textile llc,Personal and Household Goods,0
4,36539,2,Hi-Tech Textile L.L.C.,Jordan,14/10/2019,2019,NaN,hi-tech textile llc,Hi-Tech Textile LLC,106981.0,hi-tech textile llc,Personal and Household Goods,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6495,8197,36,CODEVI,Haiti,2015-10-02 00:00:00,2015,NaN,codevi,CODEVI,118360.0,codevi,Personal and Household Goods,0
6496,8262,1540,Valdor Apparel Mfg S.A.,Haiti,27/01/2015,2015,NaN,valdor apparel mfg sa,NaN,NaN,NaN,NaN,0
6497,8212,112,Casual Wear Apparel L.L.C.,Haiti,18/01/2015,2015,NaN,casual wear apparel llc,NaN,NaN,NaN,NaN,0
6498,8214,139,One World Apparel S.A.,Haiti,14/01/2015,2015,NaN,one world apparel sa,One World Apparel,72062.0,one world apparel,Personal and Household Goods,0


In [70]:
bw.to_csv("bw_result.csv",index=False)